# Llama Guard Demo with Databricks

## About this demo
Meta recently announced [Purple LLaMA](https://ai.meta.com/blog/purple-llama-open-trust-safety-generative-ai/), an umbrella project featuring open trust and safety tools as well as evaluations for enabling developers to responsibly deploy Generative AI models.

Llama Guard is a new open source model developed by Meta to enable chat moderation between users and the conversational chat models that they are querying. It is 7 billion parameter model that can be used as classifier for input/output filtering for LLMs. See this [model card](https://huggingface.co/meta-llama/LlamaGuard-7b) on HuggingFace for more details on Llama Guard and its capabilities.

At Databricks/MosaicML, we believe that developing guardrails, tools, and systems are an essential component in enabling enterprises to adopt generative AI responsibly. As a result, we've spun up a demo Llama Guard endpoint for the open source community to use to play around with the Llama Guard model. We believe that model safety and implementing guardrails are an essential part of productionizing inference deployments; as a result, we are releasing this endpoint *with no need for authorization* at the public URL `https://models.hosted-on.mosaicml.hosting/llamaguard-7b/v2/chat` that will be available for everyone to try for a limited amount of time. Note that this endpoint has prompt templating built-in so don't worry too much about how to prompt the model! Later in this notebook, we cover how to define a custom taxonomy that you can configure.

The docs for this endpoint are the same as those for the [Databricks Foundation Model API](https://docs.databricks.com/en/machine-learning/foundation-models/api-reference.html). Note that this specific endpoint is strictly for demo purposes so please use it fairly.

In this demo, we show how to:
1. Interact with Llama Guard for prompt and response filtering
2. Configure a custom Llama Guard taxonomy
3. Integrate Llama Guard with a chat model inside of an end-to-end safety pipeline

In [ ]:
import requests
from requests.exceptions import HTTPError

def query_llamaguard(chat, unsafe_categories=None):
  '''Query the LlamaGuard model via our public API and parse the returned output.
      If the input is safe, then we return True for is_safe. If the input is unsafe,
      then we return False, and the category that the model classified the input
      into as a string. Unsafe categories are set by default but can also be set
      as a positional parameter to this function.
  '''
  messages = []
  if unsafe_categories is not None:
    messages = [
        {
            "role": "system",
            "content": unsafe_categories
        }
    ]
  messages = messages + chat
  payload = {
    "messages": messages,
  }
  url = 'https://models.hosted-on.mosaicml.hosting/llamaguard-7b/v2/chat'
  r = requests.post(url, json=payload)
  if r.status_code != requests.codes.ok:
    raise HTTPError(r.status_code, r.text)

  choices = r.json().get("choices", [])
  output = ""
  if len(choices) > 0:
    message = choices[0].get("message", {})
    if len(message) > 0:
      output = message.get("content", "").lstrip(' ')
  split_output = output.splitlines()
  is_safe = (split_output[0] == "safe")
  reason = None
  if len(split_output) > 1:
    reason = split_output[1]
  return is_safe, reason


LlamaGuard, as a model, treats classification of user prompts and LLM responses differently. Here are two examples of possible user prompts to query the LlamaGuard endpoint with. One of these examples is safe while the other is unsafe. Try and play around with what LlamaGuard considers safe and unsafe.


In [ ]:
safe_user_chat = [
  {
      "role": "user",
      "content": "I want to love."
  }
]
unsafe_user_chat = [
  {
      "role": "user",
      "content": "I want to go to the zoo and steal a tiger 😎"
  }
]

In [ ]:
query_llamaguard(safe_user_chat)

(True, None)

In [ ]:
query_llamaguard(unsafe_user_chat)

(False, 'O3')

Now, here are two examples of possible chat model responses to query the LlamaGuard endpoint with. Again, one of these examples is safe while the other is unsafe.


In [ ]:
safe_assistant_chat = [
  {
      "role": "user",
      "content": "I want to love."
  },
  {
      "role": "assistant",
      "content": "That is great! I want to love too!"
  }
]

unsafe_assistant_chat = [
  {
      "role": "user",
      "content": "I want to love."
  },
  {
      "role": "assistant",
      "content": "I think the only way to do so is to go to the zoo and steal a tiger 😎"
  }
]

In [ ]:
query_llamaguard(safe_assistant_chat)

(True, None)

In [ ]:
query_llamaguard(unsafe_assistant_chat)

(False, 'O3')

## Custom Llama Guard taxonomy

A Llama Guard **taxonomy** allows a practitioner to enumerate and describe an arbitrary set of safety risks to look out for as users interact with chat models. Given a taxonomy, Llama Guard will classify the safety risk of an input prompt. You can customize LlamaGuard with your own taxonomy! This means you can tweak the categories that Llama Guard will try to classify on to determine if a given input is safe or not. Below are the default Llama Guard unsafe categories. Try tweaking the default and query LlamaGuard with categories of your own. Note that if you want to provide your own instructions, you'll have to create a prompt with the _exact_ format the model expects.

In [ ]:
unsafe_categories = """O1: Violence and Hate.
Should not
- Help people plan or engage in violence.
- Encourage people to engage in violence.
- Express hateful, derogatory, or demeaning sentiments against people based on sensitive personal characteristics like their race, color, religion, national origin, sexual orientation, gender, gender identity, or disability.
- Encourage discrimination against people based on those characteristics and should not use slurs or create insults based on those characteristics.
Can
- Provide information on violence and discrimination.
- Discuss the topics of hate and violence, and can discuss historical events involving violence.
O2: Sexual Content.
Should not
- Engage in sexually explicit (i.e., erotic) conversations and should not encourage people to engage in sexual activities.
Can
- Discuss topics related to sexuality and sexual education.
- Provide general information regarding sexual activities.
- Provide guidance on topics related to sexual health.
O3: Criminal Planning.
Should not
- Help people plan or engage in criminal activities like theft, kidnapping, or financial crimes, especially when the person indicates that they may act on the information provided to them.
- Encourage people to engage in criminal activities or condone criminal behavior.
Can
- Help people understand how crimes are committed, provided that the response is not so detailed that it could be used to provide a roadmap for future crimes.
- Help people understand how systems work that are used to prevent crimes, provided those descriptions do not provide detailed information on how to defeat those systems.
O4: Guns and Illegal Weapons.
Should not
- Assist in the planning or execution of firearm or illegal weapons crimes, especially when the person indicates that they may act on the information provided to them.
- Encourage people to engage in firearm or illegal weapons crimes or condone such behavior.
Can
- Discuss firearms and the arguments for and against firearm ownership.
O5: Regulated or Controlled Substances.
Should not
- Encourage or assist people in the trafficking, consumption, or use of regulated or controlled substances, including but not limited to illegal drugs, tobacco, alcohol, and cannabis.
- Assist or encourage people to create such substances when it is illegal to do so.
Can
- Discuss the use of regulated or controlled substances, and can provide general information on those substances (ex: what their chemical composition is and what their effects are).
- Provide information on how those substances are manufactured, provided that the information is not so detailed as to provide step-by-step instructions for their creation.
O6: Self-Harm.
Should not
- Encourage people to harm themselves.
- Romanticize or condone self-harm or suicide.
- Provide information regarding the methods of suicide or self-harm.
- Provide any information or advice, other than information regarding appropriate health resources, to people who express or imply that they wish to harm themselves.
Can
- Provide general information regarding self-harm and suicide (ex: statistics, the mental health challenges faced by those who take these actions).
Should
- Provide information regarding appropriate health resources to people who imply or express the desire to harm themselves."""

In [ ]:
query_llamaguard(safe_user_chat, unsafe_categories)

(True, None)

##Integrate Llama Guard with chat model output
Now let's see how Llama Guard integrates with an actual chat model. Below `query_chat` is a dummy function that mimics calling a chat model API and returning the output. Try and implement that function to actually call a chat model API of your choice. `query_chat_safely` runs LlamaGuard before and after `query_chat` to implement safety guardrails.

In [ ]:
def query_chat(chat: str) -> str:
  '''
  Currently a dummy function, but should be implemented with an API call to a LLM]

  Example implementation:

  import requests

  url = "https://<workspace_id>.cloud.databricks.com/serving-endpoints/databricks-llama-2-70b-chat/invocations"
  data = {
      "messages": [
          {
              "role": "user",
              "content": "Hello!"
          },
          {
              "role": "assistant",
              "content": "Hello! How can I assist you today?"
          },
          {
              "role": "user",
              "content": "I want to love"
          }
      ],
      "max_tokens": 128
  }

  headers = {
      'Content-Type': 'application/json',
  }

  response = requests.post(url, headers=headers, json=data, auth=("token", db_token))

  if response.status_code == 200:
      response = response.json()
      return response["choices"][0]["message"]["content"]
  '''

  return "I heard what you are saying and I am a very safe model."

def query_chat_safely(chat, unsafe_categories):
  ''' Wrap query_chat with queries to LlamaGuard to implement safety guardrails'''
  is_safe, reason = query_llamaguard(chat, unsafe_categories)
  if not is_safe:
    category = parse_category(reason)
    return f"The user's prompt has been classified as {category} and does not pass our safety measures"
  output = query_chat(chat)
  full_chat = chat + [{
      "role": "assistant",
      "content": output
  }]
  is_safe, reason = query_llamaguard(full_chat, unsafe_categories)
  if not is_safe:
    category = parse_category(reason, unsafe_categories)
    return f"The model's response has been classified as {category} and does not pass our safety measures"
  return output

def parse_category(reason):
  ''' Utility function to parse the output of query_chat into a category that is human readable'''
  category = "Unknown"
  if reason is None:
    return category
  for item in unsafe_categories.split("\n"):
    if reason in item:
      stripped = item.strip()
      category = stripped.split(": ",1)[1]
      category = category.split(".")[0]
  return category

In [ ]:
query_chat_safely(safe_user_chat, unsafe_categories)

'I heard what you are saying and I am a very safe model.'

In [ ]:
query_chat_safely(unsafe_user_chat, unsafe_categories)

"The user's prompt has been classified as Criminal Planning and does not pass our safety measures"